#### Imports

In [1]:
import math
import os
import random
from typing import Any, Dict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from ipywidgets import IntSlider, interact
from sklearn.metrics import classification_report, confusion_matrix
from torch.nn import Parameter
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from torchmetrics.image import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import wandb

wandb.login()

from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import RichProgressBar, ModelCheckpoint
from pytorch_lightning.tuner import Tuner

import kornia

/usr/local/lib/python3.8/dist-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cuda


wandb: Currently logged in as: nbennewiz to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.8/dist-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [2]:
# Small function for normalizing the already standardized data
"""from rich.progress import track
load_root="../QualityFiltered"
save_root="../NormalizedQualityFiltered"
for volume_name in track([path for path in os.listdir(root)]):
    #load
    volume_seq = np.load(os.path.join(load_root, volume_name))
    #min-max scaling
    volume_seq = (volume_seq - volume_seq.min())/(volume_seq.max() - volume_seq.min())
    #save
    np.save(os.path.join(save_root, volume_name), volume_seq)"""

'from rich.progress import track\nload_root="../QualityFiltered"\nsave_root="../NormalizedQualityFiltered"\nfor volume_name in track([path for path in os.listdir(root)]):\n    #load\n    volume_seq = np.load(os.path.join(load_root, volume_name))\n    #min-max scaling\n    volume_seq = (volume_seq - volume_seq.min())/(volume_seq.max() - volume_seq.min())\n    #save\n    np.save(os.path.join(save_root, volume_name), volume_seq)'

#### Dataset

In [3]:
class Dataset2D(Dataset):
    def __init__(
        self, data_paths, context_window=4, prediction_window=1, transform=None
    ):
        self.data_paths = data_paths
        self.context_window = context_window
        self.prediction_window = prediction_window
        self.transform = transform
        self.samples = []
        self.depth = None
        # For every path to a volume sequence in .npy
        for data_path in self.data_paths:
            volume_seq = np.load(data_path)
            # remember depth, all volumes need to have the same depth
            if self.depth == None or self.depth == volume_seq.shape[0]:
                self.depth = volume_seq.shape[0]
            else:
                print("The depths in one or more volumes are not the same!")
            # Convert to tensor
            volume_seq = torch.from_numpy(volume_seq)
            for h in range(volume_seq.shape[1]):
                # Generate samples
                for t in range(
                    len(volume_seq) - self.context_window - self.prediction_window + 1
                ):
                    # Input volume sequence (context_window x 1 x 256 x 256), target volume (1 x 1 x 256 x 256)
                    self.samples.append(
                        (
                            volume_seq[t : t + self.context_window, h].unsqueeze(1),
                            volume_seq[t+ self.context_window : t+ self.context_window+ self.prediction_window,h,].unsqueeze(1),
                        )
                    )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

class Dataset3D(Dataset):
    def __init__(self, data_paths, context_window=4, prediction_window=1, transform=None):
        self.data_paths = data_paths
        self.context_window = context_window
        self.prediction_window = prediction_window
        self.transform = transform
        self.samples = []
        # For every path to a volume sequence in .npy
        for data_path in self.data_paths:
            volume_seq = np.load(data_path)
            # Convert to tensor
            volume_seq = torch.from_numpy(volume_seq)

            for t in range(volume_seq.shape[0]-context_window-prediction_window+1):
                self.samples.append([volume_seq[t:t+context_window].unsqueeze(1), 
                                     volume_seq[t+context_window:t+context_window+prediction_window].unsqueeze(1)])
                # Input-shape: [T, C, D, H, W]
                # Target-shape: [T, C, D, H, W]
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]

In [4]:
class VolumeDataModule(pl.LightningDataModule):
    def __init__(self, root, batch_size=4, sequence_length=4, prediction_length=1, num_workers=0, drop_last=False, pin_memory=False, train_split=0.8, val_split=0.1, test_split=0.1):
        super().__init__()
        self.root = root
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length
        self.num_workers = num_workers
        self.drop_last = drop_last
        self.pin_memory = pin_memory
        self.train_split = train_split
        self.val_split = val_split
        self.test_split = test_split

    def setup(self, stage=None):
        data_paths = [os.path.join(self.root, path) for path in os.listdir(self.root)]
        total_size = len(data_paths)
    
        # Normalize splits if they don’t sum to 1
        split_sum = self.train_split + self.val_split + self.test_split
        if split_sum != 1.0:
            self.train_split /= split_sum
            self.val_split /= split_sum
            self.test_split /= split_sum
            print(f"Normalized splits to: train={self.train_split:.2f}, val={self.val_split:.2f}, test={self.test_split:.2f}")
    
        # Compute dataset sizes
        train_size = int(total_size * self.train_split)
        val_size = int(total_size * self.val_split)
        test_size = total_size - train_size - val_size  # Ensure all data is used
    
        # Error handling: Ensure valid split sizes
        if train_size <= 0 or val_size <= 0 or test_size <= 0:
            raise ValueError(f"Invalid dataset splits: train={train_size}, val={val_size}, test={test_size}. Check your split values.")
    
        # Perform random split
        train_paths, val_paths, test_paths = random_split(data_paths, [train_size, val_size, test_size])
    
        self.train_dataset = Dataset2D(train_paths, self.sequence_length, self.prediction_length)
        self.val_dataset = Dataset2D(val_paths, self.sequence_length, self.prediction_length)
        self.test_dataset = Dataset2D(test_paths, self.sequence_length, self.prediction_length)
    
        self.val_dataset_3d = Dataset3D(val_paths, self.sequence_length, self.prediction_length)
        self.test_dataset_3d = Dataset3D(test_paths, self.sequence_length, self.prediction_length)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory)

    def val_dataloader(self):
        return [
            DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory),
            DataLoader(self.val_dataset_3d, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory),
        ]

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory)

    def val_dataloader_3d(self):
        return DataLoader(self.val_dataset_3d, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory)
    
    def test_dataloader_3d(self):
        return DataLoader(self.test_dataset_3d, batch_size=self.batch_size, num_workers=self.num_workers, drop_last=self.drop_last, pin_memory=self.pin_memory)

    def teardown(self, stage=None):
        if stage == "fit" or stage is None:
            pass
            #print("Cleaning up after training...")

        if stage == "test" or stage is None:
            pass
            #print("Cleaning up after testing...")

        if stage == "validate" or stage is None:
            pass
            #print("Cleaning up after validation...")

        # Free memory by deleting large datasets
        del self.train_dataset
        del self.val_dataset
        del self.test_dataset
        del self.val_dataset_3d
        del self.test_dataset_3d

#### Model

In [5]:
class DoubleConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        
        if not mid_channels:
            mid_channels = out_channels
            
        self.double_conv = nn.Sequential(
            # First convolution
            nn.Conv2d(
                in_channels, 
                mid_channels,
                kernel_size=3,
                padding=1,
                bias=False  # No bias when using batch norm
            ),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            
            # Second convolution
            nn.Conv2d(
                mid_channels,
                out_channels,
                kernel_size=3,
                padding=1,
                bias=False  # No bias when using batch norm
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
        # Optional: Add residual connection if input and output channels match
        self.use_residual = in_channels == out_channels
        
    def forward(self, x):
        if self.use_residual:
            return self.double_conv(x) + x
        return self.double_conv(x)
    
class TemporalBlock(nn.Module):
    def __init__(self, channels, temporal_kernel_size=3):
        super().__init__()
        
        padding = temporal_kernel_size // 2
        
        self.temporal_conv = nn.Sequential(
            # Depthwise temporal conv
            nn.Conv1d(channels, channels,
                     kernel_size=temporal_kernel_size,
                     padding=padding,
                     groups=channels),
            nn.BatchNorm1d(channels),
            nn.ReLU(inplace=True),
            
            # Point-wise conv
            nn.Conv1d(channels, channels, kernel_size=1),
            nn.BatchNorm1d(channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # x shape: [B, C, T, H, W]
        b, c, t, h, w = x.shape
        
        # Reshape for temporal convolution
        x_temp = x.contiguous()  # Make memory contiguous
        x_temp = x_temp.permute(0, 3, 4, 1, 2)  # [B, H, W, C, T]
        x_temp = x_temp.reshape(b*h*w, c, t)
        
        # Apply temporal convolution
        x_temp = self.temporal_conv(x_temp)
        
        # Reshape back
        x_temp = x_temp.reshape(b, h, w, c, t)
        x_temp = x_temp.permute(0, 3, 4, 1, 2)  # [B, C, T, H, W]
        
        return x_temp


class UNet2DPlusTemporal(nn.Module):
    def __init__(self, input_frames=None, base_filters=32):
        super(UNet2DPlusTemporal, self).__init__()
        
        self.input_frames = input_frames
        
        # Encoder Path
        self.enc1 = DoubleConv2D(1, base_filters)
        self.pool1 = nn.MaxPool2d(2)
        
        self.enc2 = DoubleConv2D(base_filters, base_filters*2)
        self.pool2 = nn.MaxPool2d(2)
        
        # Last encoder with temporal processing
        self.enc3 = DoubleConv2D(base_filters*2, base_filters*4)
        self.temporal_enc = TemporalBlock(
            channels=base_filters*4,
            temporal_kernel_size=3
        )
        self.pool3 = nn.MaxPool2d(2)
        
        # Bottleneck with temporal processing
        self.bottleneck_spatial = DoubleConv2D(base_filters*4, base_filters*8)
        self.temporal_bottleneck = TemporalBlock(
            channels=base_filters*8,
            temporal_kernel_size=3
        )
        
        # Decoder Path
        self.upconv3 = nn.ConvTranspose2d(base_filters*8, base_filters*4, 
                                         kernel_size=2, stride=2)
        self.dec3 = DoubleConv2D(base_filters*8, base_filters*4)
        
        self.upconv2 = nn.ConvTranspose2d(base_filters*4, base_filters*2, 
                                         kernel_size=2, stride=2)
        self.dec2 = DoubleConv2D(base_filters*4, base_filters*2)
        
        self.upconv1 = nn.ConvTranspose2d(base_filters*2, base_filters, 
                                         kernel_size=2, stride=2)
        self.dec1 = DoubleConv2D(base_filters*2, base_filters)
        
        self.final_conv = nn.Conv2d(base_filters, 1, kernel_size=1)
    
    def forward(self, x):
        # x shape: [batch, time, height, width]
        b, t, c, h, w = x.shape
        assert t == self.input_frames, f"Expected {self.input_frames} frames, got {t}"
        
        # Process each time step through initial spatial encoders
        encoder_features = []
        enc3_features = []
        
        for i in range(t):
            curr_frame = x[:, i]  # [B, C, H, W]
            
            # Initial encoder path
            e1 = self.enc1(curr_frame)
            p1 = self.pool1(e1)
            
            e2 = self.enc2(p1)
            p2 = self.pool2(e2)
            
            # Store for skip connections
            encoder_features.append((e1, e2))
            
            # Last encoder
            e3 = self.enc3(p2)
            enc3_features.append(e3)
        
        # Process enc3 features temporally
        enc3_features = torch.stack(enc3_features, dim=2)  # [B, C, T, H, W]
        enc3_processed = self.temporal_enc(enc3_features)
        
        # Pool spatially after temporal processing
        b, c, t, h, w = enc3_processed.shape
        enc3_pooled = enc3_processed.contiguous()
        enc3_pooled = enc3_pooled.view(b*t, c, h, w)
        enc3_pooled = self.pool3(enc3_pooled)
        _, _, h_pooled, w_pooled = enc3_pooled.shape
        enc3_pooled = enc3_pooled.view(b, c, t, h_pooled, w_pooled)
        
        # Bottleneck processing
        bottle_features = []
        for i in range(t):
            curr_feat = enc3_pooled[:, :, i]  # [B, C, H, W]
            bottle_feat = self.bottleneck_spatial(curr_feat)
            bottle_features.append(bottle_feat)
        
        # Stack and apply temporal processing in bottleneck
        bottle_features = torch.stack(bottle_features, dim=2)  # [B, C, T, H, W]
        bottle_processed = self.temporal_bottleneck(bottle_features)
        
        # Take last temporal state for decoder
        #print(f"bottle_processed: {bottle_processed.shape}")
        #bottle_final = bottle_processed[:, :, -1]  # [B, C, H, W]
        #print(f"bottle_final: {bottle_final.shape}")
        
        # Decoder path (using last frame's encoder features)
        #e1, e2 = encoder_features[-1]
        #e3 = enc3_processed[:, :, -1]
        #print(enc3_processed.shape)
        #print(e3.shape)

        decoder_features = []
        for i in range(t):
            e1, e2 = encoder_features[i]
            bottle_final=bottle_processed[:, :, i, :, :]
            e3 = enc3_processed[:, :, i]
            d3 = self.upconv3(bottle_final)
            d3 = torch.cat([d3, e3], dim=1)
            d3 = self.dec3(d3)
            
            d2 = self.upconv2(d3)
            d2 = torch.cat([d2, e2], dim=1)
            d2 = self.dec2(d2)
            
            d1 = self.upconv1(d2)
            d1 = torch.cat([d1, e1], dim=1)
            d1 = self.dec1(d1)

            d1 = self.final_conv(d1)
            
            decoder_features.append(d1)
            
        decoder_features = torch.stack(decoder_features, dim=1)
        #decoder_features = decoder_features.squeeze(dim=2)
        
        return decoder_features

#### Pl Model

In [6]:
class HuberSSIMLoss2D(nn.Module):
    def __init__(self, alpha=0.7, delta=0.05, window_size=5, temporal_weight=0.1):
        super().__init__()
        # ------------------------------------------------------------------------------
        # ALPHA (Controls balance between Huber Loss and SSIM)
        # More alpha  → Model focuses on voxel accuracy (Huber loss)
        # Less alpha  → Model prioritizes structural similarity (SSIM)
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - 0.6 - 0.9 → Noisy data (CT/MRI with artifacts) → More Huber
        # - 0.4 - 0.7 → Sharp structures (CT/MRI edges)  → Balance of both
        # - 0.3 - 0.5 → Blurry predictions → More SSIM for finer details
        # Default: alpha = 0.7 (Strong Huber, some SSIM)
        # ------------------------------------------------------------------------------
        self.alpha = alpha  # More alpha = More reliance on Huber
        # ------------------------------------------------------------------------------
        # DELTA (Threshold where Huber Loss switches from MSE-like to MAE-like behavior)
        # - Higher delta → More sensitive to small errors (acts like MSE)
        # - Lower delta  → More resistant to outliers (acts like MAE)
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 0.05  → High-noise data (CT/MRI with artifacts) → More robust to outliers
        # - < 0.05  → Low-noise data (Well-normalized, synthetic) → More sensitive to details
        # - 0.02 - 0.05  → If predictions are too blurry
        # Default: delta = 0.05 (or dynamically adjusted per epoch)
        # ------------------------------------------------------------------------------
        self.delta = delta  # This will be dynamically updated every epoch
        # ------------------------------------------------------------------------------
        # TEMPORAL WEIGHT (Penalizes abrupt voxel intensity changes between frames)
        # - Higher value → Forces smoother transitions
        # - Lower value  → Allows more flexibility in voxel changes
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 0.2  → Strong penalty for sudden intensity jumps (Flickering reduction)
        # - 0.05 - 0.1  → Best for smoothly changing sequences (MRI, Weather, Fluids)
        # - < 0.05  → Allows more dynamic changes (if model is too rigid)
        # Default: temporal_weight = 0.1 (balanced smoothness)
        # ------------------------------------------------------------------------------
        self.temporal_weight = temporal_weight  # More weight = Stronger smoothness enforcement

        #window_size means padding
        self.ssim_module = kornia.losses.SSIMLoss(window_size=window_size, reduction="mean")

    def temporal_smoothness_loss(self, y_pred):
        #Penalizes sudden changes over time by computing L1 loss between consecutive frames.
        #So the object should remain stationary over time
        return torch.mean(torch.abs(y_pred[:, 1:] - y_pred[:, :-1]))  # Difference between t and t+1

    def forward(self, y_pred, y_true):
        B, T, C, H, W = y_pred.shape

        # Swap time and channel dimensions: [B, C*T, H, W]
        y_pred_restructured = y_pred.permute(0, 2, 1, 3, 4).reshape(B, C * T, H, W)
        y_true_restructured = y_true.permute(0, 2, 1, 3, 4).reshape(B, C * T, H, W)

        # Computes the SSIM treating time as another spatial dimension
        ssim_loss = self.ssim_module(y_pred_restructured, y_true_restructured)

        # Computes the Huber loss with the adjusted delta
        huber_loss = F.huber_loss(y_pred, y_true, delta=self.delta, reduction="mean")

        # Compute the Temporal Smoothness Loss
        temporal_loss = self.temporal_smoothness_loss(y_pred)

        # Weighted combination
        total_loss = self.alpha * huber_loss + (1 - self.alpha) * ssim_loss + self.temporal_weight * temporal_loss
        return total_loss, huber_loss, ssim_loss, temporal_loss

class HuberSSIMLoss3D(nn.Module):
    def __init__(self, alpha=0.7, delta=0.05, window_size=5, temporal_weight=0.1):
        super().__init__()
        # ------------------------------------------------------------------------------
        # ALPHA (Controls balance between Huber Loss and SSIM)
        # More alpha  → Model focuses on voxel accuracy (Huber loss)
        # Less alpha  → Model prioritizes structural similarity (SSIM)
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - 0.6 - 0.9 → Noisy data (CT/MRI with artifacts) → More Huber
        # - 0.4 - 0.7 → Sharp structures (CT/MRI edges)  → Balance of both
        # - 0.3 - 0.5 → Blurry predictions → More SSIM for finer details
        # Default: alpha = 0.7 (Strong Huber, some SSIM)
        # ------------------------------------------------------------------------------
        self.alpha = alpha  # More alpha = More reliance on Huber
        # ------------------------------------------------------------------------------
        # DELTA (Threshold where Huber Loss switches from MSE-like to MAE-like behavior)
        # - Higher delta → More sensitive to small errors (acts like MSE)
        # - Lower delta  → More resistant to outliers (acts like MAE)
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 0.05  → High-noise data (CT/MRI with artifacts) → More robust to outliers
        # - < 0.05  → Low-noise data (Well-normalized, synthetic) → More sensitive to details
        # - 0.02 - 0.05  → If predictions are too blurry
        # Default: delta = 0.05 (or dynamically adjusted per epoch)
        # ------------------------------------------------------------------------------
        self.delta = delta  # This will be dynamically updated every epoch
        # ------------------------------------------------------------------------------
        # TEMPORAL WEIGHT (Penalizes abrupt voxel intensity changes between frames)
        # - Higher value → Forces smoother transitions
        # - Lower value  → Allows more flexibility in voxel changes
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 0.2  → Strong penalty for sudden intensity jumps (Flickering reduction)
        # - 0.05 - 0.1  → Best for smoothly changing sequences (MRI, Weather, Fluids)
        # - < 0.05  → Allows more dynamic changes (if model is too rigid)
        # Default: temporal_weight = 0.1 (balanced smoothness)
        # ------------------------------------------------------------------------------
        self.temporal_weight = temporal_weight  # More weight = Stronger smoothness enforcement

        #window_size means padding
        self.ssim_module = kornia.losses.SSIM3DLoss(window_size=window_size, reduction="mean")

    def temporal_smoothness_loss(self, y_pred):
        #Penalizes sudden changes over time by computing L1 loss between consecutive frames.
        #So the object should remain stationary over time
        return torch.mean(torch.abs(y_pred[:, 1:] - y_pred[:, :-1]))  # Difference between t and t+1

    def forward(self, y_pred, y_true):
        B, T, C, D, H, W = y_pred.shape

        # Swap time and channel dimensions: [B, C*T, D, H, W]
        y_pred_restructured = y_pred.permute(0, 2, 1, 3, 4, 5).reshape(B, C * T, D, H, W)
        y_true_restructured = y_true.permute(0, 2, 1, 3, 4, 5).reshape(B, C * T, D, H, W)

        # Computes the SSIM treating time as another spatial dimension
        ssim_loss = self.ssim_module(y_pred_restructured, y_true_restructured)

        # Computes the Huber loss with the adjusted delta
        huber_loss = F.huber_loss(y_pred, y_true, delta=self.delta, reduction="mean")

        # Compute the Temporal Smoothness Loss
        temporal_loss = self.temporal_smoothness_loss(y_pred)

        # Weighted combination
        total_loss = self.alpha * huber_loss + (1 - self.alpha) * ssim_loss + self.temporal_weight * temporal_loss
        return total_loss, huber_loss, ssim_loss, temporal_loss

In [7]:
class Pl_Model(pl.LightningModule):
    def __init__(
        self,
        passed_model: nn.Module,
        config: Dict[str, Any],
    ):
        super(Pl_Model, self).__init__()
        self.passed_model = passed_model
        self.config = config
        # ------------------------------------------------------------------------------
        # DELTA FACTOR (Scales how much delta is updated per epoch)
        # - Higher delta_factor → More aggressive updates to delta
        # - Lower delta_factor  → Smoother, slower changes to delta
        # ------------------------------------------------------------------------------
        # Recommended tuning:
        # - > 1.5  → If delta is too unstable (jumps too much)
        # - 1.2 - 1.5  → Best for gradual adaptation (Default)
        # - < 1.2  → If delta changes too slowly (use for very stable datasets)
        # ------------------------------------------------------------------------------
        # Default: delta_factor = 1.2 (Balanced adaptation)
        # ------------------------------------------------------------------------------
        self.delta_factor = 1.2  # More factor = Faster delta adjustments
        self.delta = 0.05
        self.previous_delta = self.delta
        
        #speicher alle parameter ab
        self.save_hyperparameters(ignore=["passed_model"])

        # Setup training components
        self.mse_criterion = nn.MSELoss()
        self.psnr_criterion = PeakSignalNoiseRatio()
        self.huberssim2d_criterion = HuberSSIMLoss2D()
        self.huberssim3d_criterion = HuberSSIMLoss3D()
        self.huber_criterion = nn.HuberLoss()

    def forward(self, x):
        x = self.passed_model(x)
        #Tanh has a larger gradient range, reducing saturation issues compared to sigmoid.
        #Allows more stable gradient flow for deep networks.
        x = 0.5*(F.tanh(x)+1)
        return x

    def configure_optimizers(self):
        """Sets the Optimizer for the Model"""
        optimizer = optim.AdamW(
            self.parameters(), 
            lr=config['learning_rate'],
        )
        return [optimizer]

    def _calculate_loss(self, batch, mode="train"):
        """Calculates the loss for a batch in different modes (training, validation, testing)"""
        inputs, targets = batch

        #forward pass
        mse_loss = 0.0
        huber_loss = 0.0
        rmse_loss = 0.0
        ssim_loss = 0.0
        huberssim_loss = 0.0
        temporal_loss = 0.0
        psnr_loss = 0.0
        total_loss = 0.0

        #mse_loss_table = []
        for t in range(0, self.config["pred_frames"], self.config["pred_n_frames_per_step"]):
            if self.config["pred_frames"]-t<self.config["pred_n_frames_per_step"]:
                frames_this_step = self.config["pred_frames"]-t
            else:
                frames_this_step = self.config["pred_n_frames_per_step"]
            outputs = self.forward(inputs)
            #print(f"{t}:{t+frames_this_step}")
            #get only the first predicted frame
            outputs = outputs[:, :frames_this_step, :, :]
            
            #calcualte losses
            mse_loss_ = self.mse_criterion(outputs, targets[:, t:t+frames_this_step, :, :])
            rmse_loss_ = torch.sqrt(self.mse_criterion(outputs, targets[:, t:t+frames_this_step, :, :]))
            psnr_loss_ = self.psnr_criterion(outputs, targets[:, t:t+frames_this_step, :, :])
            huberssim_loss_, huber_loss_, ssim_loss_, temporal_loss_ = self.huberssim2d_criterion(outputs, targets[:, t:t+frames_this_step, :, :])
            total_loss_ = huberssim_loss_ #self.huber_criterion(outputs, targets[:, t:t+frames_this_step, :, :])  

            #mse_loss_table.append([t, mse_loss_.item()])
            
            mse_loss += mse_loss_
            huber_loss += huber_loss_
            rmse_loss += rmse_loss_
            ssim_loss += ssim_loss_
            huberssim_loss += huberssim_loss_
            temporal_loss += temporal_loss_
            psnr_loss += psnr_loss_
            total_loss += total_loss_
            
            inputs = torch.cat([inputs[:, self.config["pred_n_frames_per_step"]:, :, :], outputs], dim=1)

        #logging
        self.log(f"{mode}_mse_loss", mse_loss)
        self.log(f"{mode}_huber_loss", huber_loss)
        self.log(f"{mode}_rmse_loss", rmse_loss)
        self.log(f"{mode}_ssim_loss", ssim_loss)
        self.log(f"{mode}_huberssim_loss", huberssim_loss)
        self.log(f"{mode}_temporal_loss", temporal_loss)
        self.log(f"{mode}_psnr_loss", psnr_loss)
        self.log(f"{mode}_total_loss", total_loss, prog_bar=True)

        #table logging
        #wandb.log({"Loss per Timestep": wandb.plot.line(wandb.Table(data=mse_loss_table, columns=["Timestep", "Loss"]), "Timestep", "Loss", title="Loss per Timestep")})

        return total_loss

    def training_step(self, batch, batch_idx):
        loss = self._calculate_loss(batch, mode="train")
        return loss

    @torch.no_grad()
    def validation_step(self, batch, batch_idx, dataloader_idx):
        if dataloader_idx==1:
            inputs, targets = batch
            # iterate over depth
            outputs = []
            for d in range(inputs.shape[3]):
                inputs_ = inputs[:, :, :, d, :, :]
                #iterate over time
                outputs_d = []
                for t in range(0, inputs.shape[1], self.config["pred_n_frames_per_step"]):
                    if inputs.shape[1]-t<self.config["pred_n_frames_per_step"]:
                        frames_this_step = inputs.shape[1]-t
                    else:
                        frames_this_step = self.config["pred_n_frames_per_step"]
                    
                    outputs_t = self.forward(inputs_)
            
                    #get only the first predicted frame
                    outputs_t = outputs_t[:, :frames_this_step, :, :]
            
                    #add to depth lst
                    outputs_d.append(outputs_t)
            
                    inputs_ = torch.cat([inputs_[:, self.config["pred_n_frames_per_step"]:, :, :], outputs_t], dim=1)
            
                #concat time and add to overall lst
                outputs_d = torch.concat(outputs_d, dim=1)
                outputs.append(outputs_d)
            #stack over depth
            outputs = torch.stack(outputs, dim=3)

            #calculate losses
            mse_loss = self.mse_criterion(outputs, targets)
            rmse_loss = torch.sqrt(self.mse_criterion(outputs, targets))
            huberssim_loss, huber_loss, ssim_loss, temporal_loss = self.huberssim3d_criterion(outputs, targets)
            psnr_loss = self.psnr_criterion(outputs, targets)
            total_loss = huberssim_loss#self.huber_criterion(outputs, targets)

            #logging
            self.log(f"overall_val_mse_loss", mse_loss)
            self.log(f"overall_val_huber_loss", huber_loss)
            self.log(f"overall_val_rmse_loss", rmse_loss)
            self.log(f"overall_val_ssim_loss", ssim_loss)
            self.log(f"overall_val_huberssim_loss", huberssim_loss)
            self.log(f"overall_val_temporal_loss", temporal_loss)
            self.log(f"overall_val_psnr_loss", psnr_loss)
            self.log(f"overall_val_total_loss", total_loss)
                    
        else:
            _ = self._calculate_loss(batch, mode="val")

    @torch.no_grad()
    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")
    
    def on_train_epoch_end(self):
        #adjust delta
        val_loader = self.trainer.datamodule.val_dataloader()[0]
        all_errors = []
    
        with torch.no_grad():  
            for batch in val_loader:
                x, y = batch
                y_pred = self(x.to(self.device))
                error = torch.abs(y.to(self.device) - y_pred)
                all_errors.append(error.view(-1))
    
        all_errors = torch.cat(all_errors)
        new_delta = self.delta_factor * torch.std(all_errors).item()

        #Blend previous and new delta for smoother updates
        #is capped between 0.02 and 0.35 so that is the data is too noisy huber does not just become mse
        new_delta = min(0.5, max(0.02, 0.8 * self.previous_delta + 0.2 * new_delta))
        self.previous_delta = new_delta

        #update
        self.huberssim2d_criterion.delta = new_delta
        self.huberssim3d_criterion.delta = new_delta
    
        #logging
        self.log("delta", new_delta)
        
    
    @torch.no_grad()
    def check_losses(self, loader, mode, use_wandb=False):
        mse_loss = 0.0
        huber_loss = 0.0
        rmse_loss = 0.0
        ssim_loss = 0.0
        huberssim_loss = 0.0
        temporal_loss = 0.0
        psnr_loss = 0.0
        total_loss = 0.0
        for inputs, targets in loader:
            for t in range(self.config["pred_frames"]):
                mse_loss_ = self.mse_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :].unsqueeze(1))
                huberssim_loss_, huber_loss_, ssim_loss_, temporal_loss_ = self.huberssim2d_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :].unsqueeze(1))
                rmse_loss_ = torch.sqrt(self.mse_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :].unsqueeze(1)))
                psnr_loss_ = self.psnr_criterion(inputs[:, -1, :, :, :].unsqueeze(1), targets[:, t, :, :].unsqueeze(1))
                total_loss_ = huberssim_loss_   
                
                mse_loss += mse_loss_.item()
                huber_loss += huber_loss_.item()
                rmse_loss += rmse_loss_.item()
                ssim_loss += ssim_loss_.item()
                huberssim_loss += huberssim_loss_.item()
                temporal_loss += temporal_loss_.item()
                psnr_loss += psnr_loss_.item()
                total_loss += total_loss_.item()
                
        mse_loss = mse_loss / len(loader)
        huber_loss = huber_loss / len(loader)
        rmse_loss = rmse_loss / len(loader)
        ssim_loss = ssim_loss / len(loader)
        huberssim_loss = huberssim_loss / len(loader)
        temporal_loss = temporal_loss / len(loader)
        psnr_loss = psnr_loss / len(loader)
        total_loss = total_loss / len(loader)

        if use_wandb:
            wandb.log({f"Checked_{mode}_mse_loss": mse_loss})
            wandb.log({f"Checked_{mode}_huber_loss": huber_loss})
            wandb.log({f"Checked_{mode}_rmse_loss": rmse_loss})
            wandb.log({f"Checked_{mode}_ssim_loss": ssim_loss})
            wandb.log({f"Checked_{mode}_huberssim_loss": huberssim_loss})
            wandb.log({f"Checked_{mode}_temporal_loss": temporal_loss})
            wandb.log({f"Checked_{mode}_psnr_loss": psnr_loss})
            wandb.log({f"Checked_{mode}_total_loss": total_loss})
        
        return mse_loss, huber_loss, ssim_loss, huberssim_loss, temporal_loss, rmse_loss, psnr_loss, total_loss
        
    def log_predictions(self):
        """Log example predictions to wandb"""
        #needs to be added to other method
        if epoch % self.config['viz_interval'] == 0:
                self.log_predictions()
        #but this whole method needs to be rewritten
        self.model.eval()
        with torch.no_grad():
            # Get a batch of validation data
            data, target = next(iter(self.val_loader))
            data = data.to(self.device)
            target = target.to(self.device)
            
            # Generate predictions
            output = self.model(data)
            
            # Log images
            wandb.log({
                "predictions": wandb.Image(output[0, 0].cpu()),
                "targets": wandb.Image(target[0, 0].cpu()),
                "input_sequence": [wandb.Image(data[0, i].cpu()) for i in range(data.shape[1])]
            })

#### Config

In [8]:
config = {
    "root": "../NormalizedQualityFiltered",
    'batch_size': 2,
    'learning_rate': 0.0001,
    "num_workers": 10,#0, wenn die gpu nicht benutzt wird
    "pin_memory": True if torch.cuda.is_available() else False,#False, wenn die gpu nicht benutzt wird
    "drop_last": False,
    'epochs': 40,
    #'log_interval': 20,
    #'viz_interval': 1,
    'run_name': '2D-UNet+_temp',
    'input_frames': 9,
    "pred_frames": 9,
    "pred_n_frames_per_step": 1,
    'base_filters': 32,
    "train_split": 0.7,
    "val_split": 0.15,
    "test_split": 0.15,
}
config["run_name"] += f"_{config['pred_frames']}"
if config["pred_frames"] == config["pred_n_frames_per_step"]:
    config["run_name"] += "_singleshot"
elif config["pred_n_frames_per_step"] == 1:
    config["run_name"] += "_autoregressive"
else:
    config["run_name"] += f"_multistep_{config['pred_n_frames_per_step']}"

# Get data loaders
'''train_loader, val_loader, test_loader, val_loader_3d, test_loader_3d, train_dataset, val_dataset, test_dataset, val_dataset_3d, test_dataset_3d = get_data_loaders(
    batch_size=config['batch_size'],
    num_workers=config["num_workers"],
    pin_memory=config["pin_memory"],
    drop_last=config["drop_last"],
    sequence_length=config["input_frames"],
    prediction_length=config["pred_frames"],
)'''
dm = VolumeDataModule(
    root=config["root"],
    batch_size=config['batch_size'],
    num_workers=config["num_workers"],
    pin_memory=config["pin_memory"],
    drop_last=config["drop_last"],
    sequence_length=config["input_frames"],
    prediction_length=config["pred_frames"],
    train_split=config["train_split"],
    val_split=config["val_split"],
    test_split=config["test_split"],
)

wandb_logger = WandbLogger(entity="ChadCTP", project="perfusion-ct-prediction", name=config["run_name"])

# Initialize model for tuning
model = UNet2DPlusTemporal(
    input_frames=config['input_frames'],
    base_filters=config['base_filters'],
)

# Initialize pl_model for tuning
pl_model = Pl_Model(
    passed_model=model,
    config=config,
)

checkpoint_callback = ModelCheckpoint(
    monitor="val_total_loss/dataloader_idx_0",  
    mode="min",  
    save_top_k=1,  
    filename="best-checkpoint",
    verbose=True,
)

# Initialize trainer for tuning
trainer = pl.Trainer(
    logger=wandb_logger,
    accelerator="gpu",
    devices= [1] if torch.cuda.is_available() else None,
    max_epochs=config["epochs"],
    callbacks=[RichProgressBar(), checkpoint_callback],
    check_val_every_n_epoch=5,
)

#wandb_logger.watch(pl_model, log_graph=False)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(
    model=pl_model,
    datamodule=dm,
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃   ┃ Name                  ┃ Type                 ┃ Params ┃ Mode  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│ 0 │ passed_model          │ UNet2DPlusTemporal   │  2.0 M │ train │
│ 1 │ mse_criterion         │ MSELoss              │      0 │ train │
│ 2 │ psnr_criterion        │ PeakSignalNoiseRatio │      0 │ train │
│ 3 │ huberssim2d_criterion │ HuberSSIMLoss2D      │      0 │ train │
│ 4 │ huberssim3d_criterion │ HuberSSIMLoss3D      │      0 │ train │
│ 5 │ huber_criterion       │ HuberLoss            │      0 │ train │
└───┴───────────────────────┴──────────────────────┴────────┴───────┘

Trainable params: 2.0 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.0 M                                                                                                
Total estimated model params size (MB): 8

Output()

Epoch 4, global step 3320: 'val_total_loss/dataloader_idx_0' reached inf (best inf), saving model to './perfusion-ct-prediction/z2h7q35o/checkpoints/best-checkpoint.ckpt' as top 1
Epoch 9, global step 6640: 'val_total_loss/dataloader_idx_0' was not in top 1
Epoch 14, global step 9960: 'val_total_loss/dataloader_idx_0' was not in top 1
Epoch 19, global step 13280: 'val_total_loss/dataloader_idx_0' was not in top 1
Epoch 24, global step 16600: 'val_total_loss/dataloader_idx_0' was not in top 1
Epoch 29, global step 19920: 'val_total_loss/dataloader_idx_0' was not in top 1
Epoch 34, global step 23240: 'val_total_loss/dataloader_idx_0' was not in top 1
Epoch 39, global step 26560: 'val_total_loss/dataloader_idx_0' was not in top 1
`Trainer.fit` stopped: `max_epochs=40` reached.


In [10]:
#check and log the losses "to beat"
dm.setup()
pl_model.check_losses(dm.train_dataloader(), mode="train", use_wandb=True)
pl_model.check_losses(dm.val_dataloader()[0], mode="val", use_wandb=True)
pl_model.check_losses(dm.test_dataloader(), mode="test", use_wandb=True)

(0.004597377111765941,
 0.0007119470029513314,
 0.2184683965843808,
 nan,
 nan,
 0.18980021516204273,
 297.7571638508847,
 nan)

In [11]:
val_results = trainer.validate(pl_model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric       ┃        DataLoader 0        ┃        DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   overall_val_huber_loss   │                            │   5.5193770094774663e-05   │
│ overall_val_huberssim_loss │                            │    0.011345108039677143    │
│    overall_val_mse_loss    │                            │   0.00031943153589963913   │
│   overall_val_psnr_loss    │                            │     35.272796630859375     │
│   overall_val_rmse_loss    │                            │    0.017555508762598038    │
│   overall_val_ssim_loss    │                            │    0.03717964142560959     │
│ overall_val_temporal_loss  │                            │   0.0015258090570569038    │
│   overall_val_total_loss   │                            │    0.011345108039677143    │
│       val_huber_loss       │   0.0004967429558746517    │                            │
│     val_huberssim_loss     │            nan             │                            │
│        val_mse_loss        │   0.0028748787008225918    │                            │
│       val_psnr_loss        │     320.1243591308594      │                            │
│       val_rmse_loss        │     0.1487736999988556     │                            │
│       val_ssim_loss        │    0.16209197044372559     │                            │
│     val_temporal_loss      │            nan             │                            │
│       val_total_loss       │            nan             │                            │
└────────────────────────────┴────────────────────────────┴────────────────────────────┘

In [12]:
test_results = trainer.test(pl_model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_huber_loss      │  0.00045256895828060806   │
│    test_huberssim_loss    │            nan            │
│       test_mse_loss       │   0.0027253027074038982   │
│      test_psnr_loss       │    321.79241943359375     │
│      test_rmse_loss       │    0.1415521651506424     │
│      test_ssim_loss       │    0.14753317832946777    │
│    test_temporal_loss     │            nan            │
│      test_total_loss      │            nan            │
└───────────────────────────┴───────────────────────────┘

In [13]:
save_load_path = f"../ModelWeights/{config['run_name']}.ckpt"
trainer.save_checkpoint(save_load_path)

In [14]:
@torch.no_grad()
def overall_loss(model, loader, device):
    mse_loss = 0.0
    huber_loss = 0.0
    rmse_loss = 0.0
    #ssim_loss = 0.0
    psnr_loss = 0.0
    total_loss = 0.0
    model = model.to(device)
    for inputs, targets in loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        # iterate over depth
        outputs = []
        for d in range(inputs.shape[3]):
            inputs_ = inputs[:, :, :, d, :, :]
            #iterate over time
            outputs_d = []
            for t in range(0, model.config["pred_frames"], model.config["pred_n_frames_per_step"]):
                if model.config["pred_frames"]-t<model.config["pred_n_frames_per_step"]:
                    frames_this_step = model.config["pred_frames"]-t
                else:
                    frames_this_step = model.config["pred_n_frames_per_step"]
                
                outputs_t = model.forward(inputs_)
        
                #get only the first predicted frame
                outputs_t = outputs_t[:, :frames_this_step, :, :]

                #add to depth lst
                outputs_d.append(outputs_t)
        
                inputs_ = torch.cat([inputs_[:, model.config["pred_n_frames_per_step"]:, :, :], outputs_t], dim=1)
            #concat time and add to overall lst
            outputs_d = torch.concat(outputs_d, dim=1)
            outputs.append(outputs_d)
    
        #stack over depth
        outputs = torch.stack(outputs, dim=3)
        #print(outputs.shape)
        
        #calculate losses
        mse_loss += model.mse_criterion(outputs, targets).item()
        huber_loss += model.huber_criterion(outputs, targets).item()
        rmse_loss += torch.sqrt(model.mse_criterion(outputs, targets)).item()
        #ssim_loss = model.ssim_criterion(outputs, targets).item()
        psnr_loss += model.psnr_criterion(outputs, targets).item()
        total_loss += huber_loss

    mse_loss = mse_loss / len(loader)
    huber_loss = huber_loss / len(loader)
    rmse_loss = rmse_loss / len(loader)
    #ssim_loss = ssim_loss / len(loader)
    psnr_loss = psnr_loss / len(loader)
    total_loss = total_loss / len(loader)

    return outputs, mse_loss, huber_loss, rmse_loss, psnr_loss, total_loss

dm.setup()
_, mse_loss, huber_loss, rmse_loss, psnr_loss, total_loss = overall_loss(model=pl_model, loader=dm.test_dataloader_3d(), device=device)
mse_loss, huber_loss, rmse_loss, psnr_loss, total_loss

(0.00020112004203838297,
 0.00010056002101919149,
 0.01394939050078392,
 37.25218620300293,
 0.0006227553025382804)

In [15]:
'''# idk tuning behaves really weird and different from fit regarding memory usage
#tuning
tuner = Tuner(trainer)
tuner.scale_batch_size(pl_model, datamodule=dm, mode="binsearch")

#cleaning up
del tuner
del trainer
del pl_model
del model
gc.collect()
torch.cuda.empty_cache()
if torch.cuda.is_available():
    torch.cuda.ipc_collect()

# Reinitialize model for tuning
model = SimVP(
    shape_in=[config["input_frames"], 1, 256, 256]
)

# Reinitialize pl_model for training
pl_model = Pl_Model(
    passed_model=model,
    config=config,
)

# Reinitialize trainer for training
trainer = pl.Trainer(
    logger=wandb_logger,
    accelerator="gpu",
    devices= [2] if torch.cuda.is_available() else None,
    max_epochs=config["epochs"],
    callbacks=[RichProgressBar()],
    check_val_every_n_epoch=1,
    #enable_checkpointing=False,
)'''

'# idk tuning behaves really weird and different from fit regarding memory usage\n#tuning\ntuner = Tuner(trainer)\ntuner.scale_batch_size(pl_model, datamodule=dm, mode="binsearch")\n\n#cleaning up\ndel tuner\ndel trainer\ndel pl_model\ndel model\ngc.collect()\ntorch.cuda.empty_cache()\nif torch.cuda.is_available():\n    torch.cuda.ipc_collect()\n\n# Reinitialize model for tuning\nmodel = SimVP(\n    shape_in=[config["input_frames"], 1, 256, 256]\n)\n\n# Reinitialize pl_model for training\npl_model = Pl_Model(\n    passed_model=model,\n    config=config,\n)\n\n# Reinitialize trainer for training\ntrainer = pl.Trainer(\n    logger=wandb_logger,\n    accelerator="gpu",\n    devices= [2] if torch.cuda.is_available() else None,\n    max_epochs=config["epochs"],\n    callbacks=[RichProgressBar()],\n    check_val_every_n_epoch=1,\n    #enable_checkpointing=False,\n)'